<a href="https://colab.research.google.com/github/amandagmrsz/Projetos/blob/main/AprendendoManipularComDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, send_file
import pandas as pd
from openpyxl import load_workbook
from io import BytesIO
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

app = Flask(__name__)

# Rota para exibir o formulário
@app.route('/')
def index():
    return render_template('holerite.html')

# Rota para processar os dados do formulário e gerar o holerite
@app.route('/gerar_holerite', methods=['POST'])
def gerar_holerite():
    # Pegando os dados do formulário
    salario_bruto = float(request.form['salario_bruto'])
    desconto_inss = float(request.form['desconto_inss'])
    desconto_irrf = float(request.form['desconto_irrf'])
    beneficio = float(request.form['beneficio'])

    # Criando o DataFrame com os dados
    df = pd.DataFrame({
        'salario_bruto': [salario_bruto],
        'desconto_inss': [desconto_inss],
        'desconto_irrf': [desconto_irrf],
        'beneficio': [beneficio],
    })

    # Realizando o cálculo do salário líquido
    df['salario_liquido'] = df['salario_bruto'] - df['desconto_inss'] - df['desconto_irrf'] + df['beneficio']

    # Carregando o template de Excel (caso tenha imagens ou formatação)
    wb = load_workbook('holerite.xlsx')
    ws = wb.active

    # Preenchendo as células do Excel com os dados calculados
    for index, row in df.iterrows():
        ws['B2'] = row['salario_bruto']
        ws['B3'] = row['desconto_inss']
        ws['B4'] = row['desconto_irrf']
        ws['B5'] = row['beneficio']
        ws['B6'] = row['salario_liquido']

    # Salvar o arquivo modificado em memória (para enviar por e-mail ou download)
    output = BytesIO()
    wb.save(output)
    output.seek(0)

    # Enviar o arquivo por e-mail (exemplo de como fazer isso)
    enviar_email_com_arquivo(output)

    # Retornar o arquivo para download
    return send_file(output, as_attachment=True, download_name="holerite_simulado.xlsx")

# Função para enviar o e-mail
def enviar_email_com_arquivo(arquivo):
    from_email = "seuemail@dominio.com"
    to_email = "destinatario@dominio.com"

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = "Holerite Simulado"

    # Anexando o arquivo
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(arquivo.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment', filename="holerite_simulado.xlsx")
    msg.attach(part)

    # Enviando o e-mail (usando Gmail, por exemplo)
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(from_email, 'sua_senha')
        server.sendmail(from_email, to_email, msg.as_string())

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from io import BytesIO

# Função para gerar a imagem a partir do Excel
def gerar_imagem_excel(file_path):
    excel_data = pd.read_excel(file_path, sheet_name='Dados')  # Carregar a aba 'Dados' do Excel

    # Criação de uma figura
    fig, ax = plt.subplots(figsize=(10, 4))  # Ajuste o tamanho conforme necessário

    # Remover eixos do gráfico
    ax.axis('off')

    # Gerar a tabela no gráfico
    tabela = table(ax, excel_data, loc='center', colWidths=[0.1]*len(excel_data.columns))

    # Ajuste da tabela
    tabela.auto_set_font_size(False)
    tabela.set_fontsize(8)
    tabela.scale(1.2, 1.2)

    # Salvar a imagem em um buffer de memória
    img_buffer = BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)

    return img_buffer

# Função para enviar a imagem por e-mail
def enviar_email_com_imagem(img_buffer, to_email):
    from_email = 'seuemail@dominio.com'
    password = 'sua_senha'

    # Configurações do servidor SMTP
    server = smtplib.SMTP('smtp.dominio.com', 587)
    server.starttls()
    server.login(from_email, password)

    # Criando o e-mail
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = 'Imagem do Holerite Atualizado'

    # Anexando a imagem
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(img_buffer.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment', filename='holerite_imagem.png')
    msg.attach(part)

    # Enviar o e-mail
    server.sendmail(from_email, to_email, msg.as_string())
    server.quit()

# Exemplo de uso
file_path = 'C:/caminho/para/seu/holerite.xlsx'  # Caminho do arquivo Excel
img_buffer = gerar_imagem_excel(file_path)
enviar_email_com_imagem(img_buffer, 'destinatario@dominio.com')
